In [ ]:
CONFIG_FILE_PATH_PREFIX = 'cfg/'
CONFIG_TMP_FILE_PATH_PREFIX = 'cfg/tmp/'
CONFIG_BACKUP_FILE_PATH_PREFIX = 'cfg/backup/'

OUTPUT_FILE_PATH_PREFIX = 'output/'
OUTPUT_TMP_FILE_PATH_PREFIX = 'output/tmp/'

CONFIG_NUMBER = 1000

In [ ]:
#!pip install Faker

In [ ]:
#!pip install requests beautifulsoup4 lxml

In [ ]:
import sys
import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType
spark = SparkSession \
    .builder \
    .appName("Python Spark Customer Fact Generator") \
    .getOrCreate()

In [ ]:
##Reading the config from file for max id
import os.path
idConfig = {
    "customerrecordid": 10000000,
    "customerid": 10000000,
    "passpost_identitynumber": 111111111,
    "govementid_identitynumber": 111111111,
    "company_identitynumber": 111111111,
    "chiefaccountantid": 10000000,
    "oldercustomersaveid": 10000000,
    "mainaccountid": 10000000,
    "taxcode": 0,
}
print('-----------------')
for file in idConfig.keys():
    file_exist = os.path.exists(CONFIG_FILE_PATH_PREFIX+file)
    if (file_exist):
        print(file + ' is exist')
        print('begin loading' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"r")
        idConfig[file] = int(realFile.readlines()[0])
        realFile.close()
    else:
        print(file + 'is not exist')
        print('begin to creating ' + file)
        realFile = open(CONFIG_FILE_PATH_PREFIX+file,"w+")
        realFile.seek(0)
        realFile.write(str(idConfig[file]))
        realFile.close()
    print('-----------------')
    print('begin to creating ' + 'default' + file )
    realFile = open(CONFIG_FILE_PATH_PREFIX+'default'+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()

In [ ]:
import random
import faker
from datetime import datetime
import re
import requests
from bs4 import BeautifulSoup

class CustomerObject:
    customerType = {
        1: "InvidiualUser",
        2: "LinkedCompanyUser",
        3: "LinkedInvidiualUser"
    }
    gender = {
        1: "male",
        2: "female"
    }
    personal_identityType = {
        1: "Passpost",
        2: "Govement ID",
    }
    occupationtypetitle = {
        1: "Officer",
        2: "Docter, Pharmacist, nurse",
        3: "Civil Servent",
        4: "Engineer, Construction worker, Mechanical Engineer",
        5: "Armed Forces",
        6: "Freelancer",
        7: "Pupil, Student",
        8: "Other"
    }
    jobtitle = {
        1: "Staff",
        2: "Manager, Supervisor",
        3: "Director, Senior Manager",
        4: "Business Owner"
    }
    fake = faker.Faker()

    def __init__(self, customerType):
        if(customerType < min(list(CustomerObject.customerType.keys())) or 
           customerType > min(list(CustomerObject.customerType.keys()))):
            __customerType = random.choice(list(CustomerObject.customerType.keys()))
        else:
            __customerType = customerType;
        self.customerrecordid = idConfig["customerrecordid"]
        idConfig["customerrecordid"] += 1
        self.customerid = idConfig["customerid"]
        idConfig["customerid"] += 1
        self.customertype = CustomerObject.customerType[__customerType]
        __gender = random.choice(list(CustomerObject.gender.keys()))
        if (__gender == 1):
            self.name = CustomerObject.fake.name_male()
        elif (__gender == 2):
            self.name = CustomerObject.fake.name_female()
        
        self.birthday = CustomerObject.fake.date()
        self.gender = CustomerObject.gender[__gender]
        self.nationality = CustomerObject.fake.country()
        self.residentstatus = random.choice([0,1])
        if (self.residentstatus == 0):
            self.residentperiodinvn = CustomerObject.fake.date_time_this_decade().strftime("%Y-%m-%d")
        else:
            self.residentperiodinvn = ""
        __personal_identitytype = random.choice(list(CustomerObject.personal_identityType.keys()))
        self.identitytype = CustomerObject.personal_identityType[__personal_identitytype]
        self.issueby = ""
        if (__personal_identitytype == 1):
            self.identitynumber = idConfig["passpost_identitynumber"]
            idConfig["passpost_identitynumber"] += 1
        elif (__personal_identitytype == 2):
            self.identitynumber = idConfig["govementid_identitynumber"]
            idConfig["govementid_identitynumber"] += 1
        
        self.issuedate = CustomerObject.fake.date_time_between(
            start_date=datetime.strptime(self.birthday, "%Y-%m-%d")).strftime("%Y-%m-%d")
        self.expdate = CustomerObject.fake.date_time_between(
            start_date=datetime.strptime(self.issuedate, "%Y-%m-%d")).strftime("%Y-%m-%d")
        self.taxcode = idConfig["taxcode"]
        idConfig["taxcode"] += 1
        self.occupationtypetitle = random.choice(list(CustomerObject.occupationtypetitle.values()))
        if (self.occupationtypetitle == "Other"):
            self.otheroccupationdescriptions = ""
        else:
            self.otheroccupationdescriptions = ""
        self.jobtitle = random.choice(list(CustomerObject.jobtitle.values()))
        if (self.jobtitle == "Other"):
            self.otherjobdescriptions = CustomerObject.fake.job()
        else:
            self.otherjobdescriptions = ""
        self.permanentaddvn = CustomerObject.fake.address()
        self.overseaadd = CustomerObject.fake.address()
        self.currentaddvn = CustomerObject.fake.address()
        self.timeatcurrentaddress = random.randrange(20)
        self.email = CustomerObject.fake.email()
        self.phonenumber = CustomerObject.fake.phone_number()
        if (__customerType == 2):
            self.linkedcompany = random.randrange(idConfig["customerid"])
            self.linkeduser = ""
        elif (__customerType == 3):
            self.linkedcompany = ""
            self.linkeduser = random.randrange(idConfig["customerid"])
        else:
            self.linkedcompany = ""
            self.linkeduser = ""
        self.updatedate = CustomerObject.fake.date_time_between(
            start_date=datetime.strptime(self.birthday, "%Y-%m-%d")).strftime("%Y-%m-%d, %H:%M:%S")
        
    def toList(self):
        return [Row(
            self.customerrecordid,
            self.customerid,
            self.customertype,
            self.name,
            self.birthday,
            self.gender,
            self.nationality,
            self.residentstatus,
            self.residentperiodinvn,
            self.identitytype,
            self.issueby,
            self.identitynumber,
            self.issuedate,
            self.expdate,
            self.taxcode,
            self.occupationtypetitle,
            self.otheroccupationdescriptions,
            self.jobtitle,
            self.otherjobdescriptions,
            self.permanentaddvn,
            self.overseaadd,
            self.currentaddvn,
            self.timeatcurrentaddress,
            self.email,
            self.phonenumber,
            self.linkedcompany,
            self.updatedate
            )]
    def getVNaddress(self):
        URL = "https://www.fakeaddresscopy.com/Vietnam/"
        resp = requests.get(URL)
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.content, "html.parser")
            results = soup.find(id="copyInput5")
            return results.get('value')
        else:
            return ""

In [ ]:
FactCustomer = []
for x in range(CONFIG_NUMBER):
    Customer = CustomerObject(0).toList()
    FactCustomer = FactCustomer + Customer

In [ ]:
FactCustomerColumns = StructType([
    StructField("customerrecordid", StringType(), True),
    StructField("customerid", StringType(), True),
    StructField("customertype", StringType(), True),
    StructField("name", StringType(), True),
    StructField("birthday", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("nationality", StringType(), True),
    StructField("residentstatus", StringType(), True),
    StructField("residentperiodinvn", StringType(), True),
    StructField("identitytype", StringType(), True),
    StructField("issueby", StringType(), True),
    StructField("identitynumber", StringType(), True),
    StructField("issuedate", StringType(), True),
    StructField("expdate", StringType(), True),
    StructField("taxcode", StringType(), True),
    StructField("occupationtypetitle", StringType(), True),
    StructField("otheroccupationdescriptions", StringType(), True),
    StructField("jobtitle", StringType(), True),
    StructField("otherjobdescriptions", StringType(), True),
    StructField("permanentaddvn", StringType(), True),
    StructField("overseaadd", StringType(), True),
    StructField("currentaddvn", StringType(), True),
    StructField("timeatcurrentaddress", StringType(), True),
    StructField("email", StringType(), True),
    StructField("phonenumber", StringType(), True),
    StructField("linkedcompany", StringType(), True),
    StructField("updatedate", StringType(), True)
    ])

In [ ]:
MainDf = spark.createDataFrame(data=FactCustomer, schema = FactCustomerColumns)

In [ ]:
from datetime import datetime
now = datetime.now()
fileName = 'FactCustomer ' + now.strftime("%Y-%m-%d %H-%M-%S-%f") 

In [ ]:
print(fileName)

In [ ]:
MainDf.write.mode('overwrite').option("header","true").csv(OUTPUT_TMP_FILE_PATH_PREFIX + fileName)

In [ ]:
for file in idConfig.keys():
    print('begin to rewrite ' + file)
    realFile = open(CONFIG_TMP_FILE_PATH_PREFIX+file,"w+")
    realFile.seek(0)
    realFile.write(str(idConfig[file]))
    realFile.close()
    print('-----------------')

In [ ]:
import shutil
for file in idConfig.keys():
    shutil.move(CONFIG_FILE_PATH_PREFIX+file, CONFIG_BACKUP_FILE_PATH_PREFIX+file)
    shutil.move(CONFIG_TMP_FILE_PATH_PREFIX+file, CONFIG_FILE_PATH_PREFIX+file)
shutil.move(OUTPUT_TMP_FILE_PATH_PREFIX + fileName, OUTPUT_FILE_PATH_PREFIX+fileName)